<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/Pruebas_de_Diferentes_Visualizaciones_PF_MI_MD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carga de datos desde la PC

In [2]:
import pandas as pd
from IPython.display import display, clear_output
import ipywidgets as widgets
import io

# --- WIDGET DE SUBIDA DE ARCHIVOS ---
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightgreen'}
)

output = widgets.Output()

# Variables globales para guardar los DataFrames
df_mi = pd.DataFrame()
df_md = pd.DataFrame()

def cargar_archivos(change):
    global df_mi, df_md
    with output:
        clear_output(wait=True)
        archivos = upload_widget.value

        if not archivos:
            print("⚠️ No se subió ningún archivo.")
            return

        for nombre_archivo, archivo_info in archivos.items():
            try:
                contenido = archivo_info['content']
                extension = nombre_archivo.split('.')[-1].lower()

                if extension == 'csv':
                    df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                elif extension == 'xlsx':
                    df = pd.read_excel(io.BytesIO(contenido))
                else:
                    print(f"❌ Formato no compatible: {nombre_archivo}")
                    continue

                df.columns = df.columns.str.strip()

                # Detección de margen a partir del nombre
                if 'mi' in nombre_archivo.lower():
                    df_mi = pd.concat([df_mi, df], ignore_index=True)
                    print(f"✅ Cargado en df_mi: {nombre_archivo}")
                elif 'md' in nombre_archivo.lower():
                    df_md = pd.concat([df_md, df], ignore_index=True)
                    print(f"✅ Cargado en df_md: {nombre_archivo}")
                else:
                    print(f"⚠️ No se detectó margen en el nombre: {nombre_archivo}")

            except Exception as e:
                print(f"❌ Error al procesar {nombre_archivo}: {e}")

        # Vista previa
        if not df_mi.empty:
            print("\n📄 Vista previa df_mi:")
            display(df_mi.head())
        if not df_md.empty:
            print("\n📄 Vista previa df_md:")
            display(df_md.head())

upload_widget.observe(cargar_archivos, names='value')

# --- MOSTRAR WIDGET E INTERFAZ ---
display(upload_widget)
display(output)


FileUpload(value={}, accept='.csv,.xlsx', description='Subir archivos', multiple=True, style=ButtonStyle(butto…

Output()

#Instalación de Dependencias para las Visualizaciones

In [3]:
%pip install ipywidgets plotly
from google.colab import output as colab_output
colab_output.enable_custom_widget_manager()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 43.4 MB/s eta 0:00:00


In [4]:
%pip install -U kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 9.4 MB/s eta 0:00:00


#Visualizaciones de los Puntos Fijos de Márgen Derecha  / Izquierda (Claude)

In [4]:
# === VISUALIZACIÓN INTERACTIVA UNIFICADA PARA PUNTOS FIJOS (MI y MD) ===

import pandas as pd

try:
    import plotly.graph_objects as go
    import plotly.colors as pc
    import ipywidgets as widgets
    from IPython.display import display, clear_output, HTML
    import importlib.util
    import os
except ImportError:
    raise ImportError("Ejecuta: pip install plotly ipywidgets")

# ---- VERIFICAR QUE LOS DATAFRAMES ESTÉN CARGADOS ----
try:
    df_mi
    df_md
except NameError:
    print("❌ Las variables 'df_mi' y/o 'df_md' no están definidas. Por favor, ejecuta primero la carga de datos.")
else:
    # ---- CONFIGURACIÓN INICIAL ----
    datasets = {
        "Margen Izquierda (MI)": df_mi.copy(),
        "Margen Derecha (MD)": df_md.copy()
    }

    # ---- OPCIONES DE CONFIGURACIÓN ----
    estilos_grafico = [
        "Curvas suaves (spline)",
        "Líneas rectas",
        "Puntos",
        "Líneas + Puntos",
        "Área apilada",
        "Área + Líneas",
        "Área + Líneas + Puntos"
    ]

    tamanios_imagen = {
        "Pequeño (600x400)": (600, 400),
        "Mediano (900x500)": (900, 500),
        "Grande (1200x700)": (1200, 700),
        "Extra grande (1600x1000)": (1600, 1000)
    }

    grosores = {
        "Fino (1px)": 1,
        "Normal (2px)": 2,
        "Medio (4px)": 4,
        "Grueso (7px)": 7,
        "Extra grueso (10px)": 10
    }

    paletas = {
        "Plotly": pc.qualitative.Plotly,
        "D3": pc.qualitative.D3,
        "Viridis": pc.sequential.Viridis,
        "Cividis": pc.sequential.Cividis,
        "Inferno": pc.sequential.Inferno,
        "Pastel": pc.qualitative.Pastel,
        "Bold": pc.qualitative.Bold,
        "Set1": pc.qualitative.Set1,
        "Dark2": pc.qualitative.Dark2
    }

    # ---- FUNCIÓN PARA ACTUALIZAR OPCIONES DEPENDIENTES ----
    def actualizar_opciones(change=None):
        margen_seleccionado = margen_dropdown.value
        df_actual = datasets[margen_seleccionado]

        # Procesar fechas
        df_actual['FECHA'] = pd.to_datetime(df_actual['FECHA'], dayfirst=True, errors='coerce')

        # Actualizar variables disponibles
        columnas_excluidas = ['FECHA', 'INSTRUMENTO', 'MARGEN']
        variables = [col for col in df_actual.select_dtypes(include='number').columns if col not in columnas_excluidas]
        variable_dropdown.options = variables
        if variables:
            variable_dropdown.value = variables[0]

        # Actualizar puntos fijos
        puntos_fijos = sorted(df_actual['INSTRUMENTO'].dropna().unique())
        opciones_puntos = ["Todos"] + list(puntos_fijos)
        punto_dropdown.options = opciones_puntos
        punto_dropdown.value = "Todos"

        # Actualizar años
        anios = sorted(df_actual['FECHA'].dt.year.dropna().unique())
        opciones_anios = ["Todos"] + [str(a) for a in anios]
        anio_dropdown.options = opciones_anios
        anio_dropdown.value = "Todos"

    # ---- CREAR WIDGETS ----
    margen_dropdown = widgets.Dropdown(
        options=list(datasets.keys()),
        value=list(datasets.keys())[0],
        description="Margen:"
    )

    punto_dropdown = widgets.Dropdown(
        options=["Todos"],
        value="Todos",
        description="Punto Fijo:"
    )

    variable_dropdown = widgets.Dropdown(
        options=[],
        description="Variable:"
    )

    estilo_dropdown = widgets.Dropdown(
        options=estilos_grafico,
        value="Curvas suaves (spline)",
        description="Estilo gráfica:"
    )

    anio_dropdown = widgets.Dropdown(
        options=["Todos"],
        value="Todos",
        description="Año:"
    )

    tamanio_dropdown = widgets.Dropdown(
        options=list(tamanios_imagen.keys()),
        value="Mediano (900x500)",
        description="Tamaño:"
    )

    grosor_dropdown = widgets.Dropdown(
        options=list(grosores.keys()),
        value="Normal (2px)",
        description="Grosor línea:"
    )

    paleta_dropdown = widgets.Dropdown(
        options=list(paletas.keys()),
        value="Plotly",
        description="Paleta colores:"
    )

    boton = widgets.Button(description="Graficar", button_style="success")
    output = widgets.Output()
    output_guardar = widgets.Output()

    # ---- CONFIGURAR DEPENDENCIAS ----
    margen_dropdown.observe(actualizar_opciones, names='value')

    # ---- INICIALIZAR OPCIONES ----
    actualizar_opciones()

    # ---------- BLOQUE PARA GUARDAR LA GRÁFICA -------------
    formatos = {
        "PNG": ".png",
        "JPEG": ".jpg",
        "SVG": ".svg",
        "PDF": ".pdf",
        "HTML": ".html"
    }

    formato_dropdown = widgets.Dropdown(
        options=list(formatos.keys()),
        value="PNG",
        description="Formato:"
    )

    ruta_text = widgets.Text(
        value="grafica_exportada",
        description="Ruta y nombre:",
        placeholder="ej: ./carpeta/mi_grafica"
    )

    boton_guardar = widgets.Button(description="Guardar gráfica", button_style="info")

    def guardar_grafica(b=None):
        with output_guardar:
            clear_output(wait=True)
            ext = formatos[formato_dropdown.value]
            ruta_archivo = ruta_text.value
            if not ruta_archivo.lower().endswith(ext):
                ruta_archivo += ext
            if 'fig' not in globals() or not isinstance(fig, go.Figure):
                print("❌ Primero debes generar una gráfica.")
                return
            try:
                if formato_dropdown.value in ["PNG", "JPEG", "SVG", "PDF"]:
                    if importlib.util.find_spec("kaleido") is None:
                        print("❌ Para guardar como imagen/vector/pdf, instala 'kaleido':\n%pip install -U kaleido")
                        return
                    fig.write_image(ruta_archivo, format=formato_dropdown.value.lower())
                elif formato_dropdown.value == "HTML":
                    fig.write_html(ruta_archivo)
                else:
                    print("❌ Tipo de archivo no soportado.")
                    return
            except Exception as e:
                print("❌ Error al guardar la gráfica:", e)
                return

            print(f"✅ Gráfica guardada en: {os.path.abspath(ruta_archivo)}")
            if os.path.exists(ruta_archivo):
                if 'google.colab' in str(get_ipython()):
                    from google.colab import files
                    files.download(ruta_archivo)
                else:
                    ruta_abs = os.path.abspath(ruta_archivo)
                    display(HTML(f'<a href="file://{ruta_abs}" target="_blank">Descargar archivo</a>'))

    boton_guardar.on_click(guardar_grafica)
    controles_guardar = widgets.HBox([formato_dropdown, ruta_text])

    # ---------- FIN BLOQUE GUARDADO ------------------------

    # ---- FUNCIÓN PRINCIPAL DE GRAFICACIÓN ----
    def graficar(b=None):
        global fig
        with output:
            clear_output(wait=True)

            # Obtener valores de los widgets
            margen_seleccionado = margen_dropdown.value
            variable = variable_dropdown.value
            estilo = estilo_dropdown.value
            punto = punto_dropdown.value
            anio = anio_dropdown.value
            ancho, alto = tamanios_imagen[tamanio_dropdown.value]
            grosor = grosores[grosor_dropdown.value]
            paleta = paletas[paleta_dropdown.value]

            # Obtener el DataFrame correspondiente
            df = datasets[margen_seleccionado].copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')

            # Filtrar datos
            df_plot = df.dropna(subset=['FECHA', 'INSTRUMENTO', variable])

            if anio != "Todos":
                df_plot = df_plot[df_plot['FECHA'].dt.year == int(anio)]
            if punto != "Todos":
                df_plot = df_plot[df_plot['INSTRUMENTO'] == punto]

            if df_plot.empty:
                print("No hay datos para graficar con la selección actual.")
                return

            # Crear la figura
            fig = go.Figure()
            instrumentos = sorted(df_plot['INSTRUMENTO'].unique())
            color_map = {pf: paleta[i % len(paleta)] for i, pf in enumerate(instrumentos)}

            for pf in instrumentos:
                data_pf = df_plot[df_plot['INSTRUMENTO'] == pf]
                line_args = dict(width=grosor, color=color_map[pf])
                marker_args = dict(color=color_map[pf])

                if estilo == "Curvas suaves (spline)":
                    fig.add_trace(go.Scatter(
                        x=data_pf['FECHA'],
                        y=data_pf[variable],
                        mode="lines",
                        name=pf,
                        line_shape="spline",
                        line=line_args
                    ))
                elif estilo == "Líneas rectas":
                    fig.add_trace(go.Scatter(
                        x=data_pf['FECHA'],
                        y=data_pf[variable],
                        mode="lines",
                        name=pf,
                        line_shape="linear",
                        line=line_args
                    ))
                elif estilo == "Puntos":
                    fig.add_trace(go.Scatter(
                        x=data_pf['FECHA'],
                        y=data_pf[variable],
                        mode="markers",
                        name=pf,
                        marker=marker_args
                    ))
                elif estilo == "Líneas + Puntos":
                    fig.add_trace(go.Scatter(
                        x=data_pf['FECHA'],
                        y=data_pf[variable],
                        mode="lines+markers",
                        name=pf,
                        line_shape="linear",
                        line=line_args,
                        marker=marker_args
                    ))
                elif estilo == "Área apilada":
                    fig.add_trace(go.Scatter(
                        x=data_pf['FECHA'],
                        y=data_pf[variable],
                        mode="lines",
                        name=pf,
                        stackgroup='one',
                        line_shape="linear",
                        line=line_args,
                        marker=marker_args
                    ))
                elif estilo == "Área + Líneas":
                    fig.add_trace(go.Scatter(
                        x=data_pf['FECHA'],
                        y=data_pf[variable],
                        mode="lines",
                        name=pf,
                        fill="tozeroy",
                        line_shape="linear",
                        line=line_args,
                        marker=marker_args
                    ))
                elif estilo == "Área + Líneas + Puntos":
                    fig.add_trace(go.Scatter(
                        x=data_pf['FECHA'],
                        y=data_pf[variable],
                        mode="lines+markers",
                        name=pf,
                        fill="tozeroy",
                        line_shape="linear",
                        line=line_args,
                        marker=marker_args
                    ))

            fig.update_layout(
                width=ancho,
                height=alto,
                title=f"{margen_seleccionado}: {variable} en función del tiempo por Punto Fijo (PF)",
                xaxis_title="Fecha",
                yaxis_title=variable,
                legend_title="INSTRUMENTO",
                hovermode="x unified"
            )
            fig.show()

    # ---- INTERFAZ DE USUARIO ----
    # Título principal
    titulo_html = "<h2 style='color:#1866a3; margin-bottom: 5px'>Visualización Interactiva - Puntos Fijos (MI y MD)</h2>"
    display(HTML(titulo_html))

    # Primera fila de selectores
    selectores_fila1 = widgets.HBox([
        margen_dropdown, punto_dropdown, variable_dropdown, estilo_dropdown
    ])
    display(selectores_fila1)

    # Segunda fila de selectores
    selectores_fila2 = widgets.HBox([
        anio_dropdown, tamanio_dropdown, grosor_dropdown, paleta_dropdown
    ])
    display(selectores_fila2)

    # Botón graficar
    display(boton)

    # Salida de la gráfica
    display(output)

    # Controles de guardado
    display(controles_guardar)
    display(boton_guardar)
    display(output_guardar)

    # Conectar evento del botón
    boton.on_click(graficar)
    boton_guardar.on_click(guardar_grafica)

KeyError: 'FECHA'

#Visualizaciones de los Puntos Fijos de Márgen Derecha  / Izquierda (Deepseek)

In [7]:
# ===== VISUALIZACIÓN UNIFICADA PUNTOS FIJOS - MÁRGENES IZQUIERDA/DERECHA =====
import pandas as pd
import plotly.graph_objects as go
import plotly.colors as pc
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import importlib.util
import os
import io

# --- Configuración inicial ---
MARGENES = {
    "Margen Izquierda (MI)": "df_mi",
    "Margen Derecha (MD)": "df_md"
}

# --- Widget Principal de Selección ---
margen_dropdown = widgets.Dropdown(
    options=list(MARGENES.keys()),
    value=list(MARGENES.keys())[0],
    description="Seleccionar margen:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='400px')
)

# --- Variables Globales ---
df_actual = pd.DataFrame()
titulo_dataset = ""
fig = None

# --- Componentes Reutilizables ---
def crear_dropdown(opciones, descripcion, valor_inicial=None):
    return widgets.Dropdown(
        options=opciones,
        value=valor_inicial or opciones[0] if opciones else None,
        description=descripcion,
        disabled=not opciones,
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='300px')
    )

# Configuración de estilos
ESTILOS_GRAFICO = [
    "Curvas suaves (spline)", "Líneas rectas", "Puntos",
    "Líneas + Puntos", "Área apilada", "Área + Líneas", "Área + Líneas + Puntos"
]

TAMANIOS_IMAGEN = {
    "Pequeño (600x400)": (600, 400),
    "Mediano (900x500)": (900, 500),
    "Grande (1200x700)": (1200, 700),
    "Extra grande (1600x1000)": (1600, 1000)
}

GROSORES = {
    "Fino (1px)": 1, "Normal (2px)": 2, "Medio (4px)": 4,
    "Grueso (7px)": 7, "Extra grueso (10px)": 10
}

PALETAS = {
    "Plotly": pc.qualitative.Plotly,
    "D3": pc.qualitative.D3,
    "Viridis": pc.sequential.Viridis,
    "Cividis": pc.sequential.Cividis,
    "Inferno": pc.sequential.Inferno,
    "Pastel": pc.qualitative.Pastel,
    "Bold": pc.qualitative.Bold,
    "Set1": pc.qualitative.Set1,
    "Dark2": pc.qualitative.Dark2
}

FORMATOS_GUARDADO = {
    "PNG": ".png", "JPEG": ".jpg", "SVG": ".svg",
    "PDF": ".pdf", "HTML": ".html"
}

# --- Widgets Interactivos ---
punto_dropdown = crear_dropdown([], "Punto Fijo:")
variable_dropdown = crear_dropdown([], "Variable:")
estilo_dropdown = crear_dropdown(ESTILOS_GRAFICO, "Estilo gráfico:", "Curvas suaves (spline)")
anio_dropdown = crear_dropdown([], "Año:")
tamanio_dropdown = crear_dropdown(list(TAMANIOS_IMAGEN.keys()), "Tamaño:", "Mediano (900x500)")
grosor_dropdown = crear_dropdown(list(GROSORES.keys()), "Grosor línea:", "Normal (2px)")
paleta_dropdown = crear_dropdown(list(PALETAS.keys()), "Paleta colores:", "Plotly")
formato_dropdown = crear_dropdown(list(FORMATOS_GUARDADO.keys()), "Formato guardado:", "PNG")

ruta_text = widgets.Text(
    value="grafica_presas",
    description="Nombre archivo:",
    placeholder="ej: analisis_margen_izquierda",
    layout=widgets.Layout(width='400px')
)

boton_graficar = widgets.Button(
    description="Generar Gráfico",
    button_style='success',
    icon='chart-line',
    layout=widgets.Layout(width='200px')
)

boton_guardar = widgets.Button(
    description="Exportar Gráfico",
    button_style='info',
    icon='save',
    layout=widgets.Layout(width='200px')
)

# --- Contenedores Visuales ---
output_grafico = widgets.Output()
output_mensajes = widgets.Output()

# --- Estilo CSS Personalizado ---
estilo_css = """
<style>
.widget-label { font-weight: bold !important; color: #1866a3 !important; }
h2.encabezado { color: #1e3c72; border-bottom: 2px solid #4a90e2; padding-bottom: 5px; }
.area-grafico { border: 1px solid #e0e0e0; border-radius: 5px; padding: 15px; margin-top: 10px; }
.controles { background-color: #f8f9fa; border-radius: 5px; padding: 15px; margin-bottom: 15px; }
</style>
"""
display(HTML(estilo_css))

# --- Cabecera Informativa ---
display(HTML(
    "<h2 class='encabezado'>Análisis de Estabilidad en Presas Patagonia - Monitoreo de Puntos Fijos</h2>"
    "<p>Seleccione margen, instrumentos y parámetros de visualización. Datos actualizados al 2025-06-15</p>"
))

# --- Función para Actualizar Datasets ---
def actualizar_dataset(change):
    global df_actual, titulo_dataset
    with output_mensajes:
        clear_output(wait=True)
        margen_seleccionado = margen_dropdown.value
        df_name = MARGENES[margen_seleccionado]

        try:
            df_actual = globals()[df_name].copy()
            titulo_dataset = margen_seleccionado

            # Procesamiento de fechas
            df_actual['FECHA'] = pd.to_datetime(
                df_actual['FECHA'],
                dayfirst=True,
                errors='coerce'
            )

            # Actualizar dropdowns
            puntos_fijos = ["Todos"] + sorted(df_actual['INSTRUMENTO'].dropna().unique().tolist())
            punto_dropdown.options = puntos_fijos
            punto_dropdown.disabled = False

            columnas_numericas = [col for col in df_actual.select_dtypes(include='number').columns
                                 if col not in ['FECHA', 'INSTRUMENTO', 'MARGEN']]
            variable_dropdown.options = columnas_numericas
            variable_dropdown.disabled = not columnas_numericas

            anios = ["Todos"] + [str(a) for a in sorted(df_actual['FECHA'].dt.year.dropna().unique())]
            anio_dropdown.options = anios
            anio_dropdown.disabled = False

            print(f"✅ Dataset cargado: {margen_seleccionado} ({len(df_actual)} registros)")

        except Exception as e:
            print(f"❌ Error al cargar dataset: {str(e)}")

# --- Función de Graficado ---
def generar_grafico(b=None):
    global fig
    with output_grafico:
        clear_output(wait=True)

        if df_actual.empty:
            print("⚠️ Primero seleccione un margen válido")
            return

        # Recuperar parámetros
        variable = variable_dropdown.value
        punto = punto_dropdown.value
        anio = anio_dropdown.value
        estilo = estilo_dropdown.value
        ancho, alto = TAMANIOS_IMAGEN[tamanio_dropdown.value]
        grosor = GROSORES[grosor_dropdown.value]
        paleta = PALETAS[paleta_dropdown.value]

        # Filtrar dataset
        df_filtrado = df_actual.dropna(subset=['FECHA', 'INSTRUMENTO', variable]).copy()

        if anio != "Todos":
            df_filtrado = df_filtrado[df_filtrado['FECHA'].dt.year == int(anio)]
        if punto != "Todos":
            df_filtrado = df_filtrado[df_filtrado['INSTRUMENTO'] == punto]

        if df_filtrado.empty:
            print("⛔ No hay datos disponibles con los filtros seleccionados")
            return

        # Crear figura
        fig = go.Figure()
        instrumentos = sorted(df_filtrado['INSTRUMENTO'].unique())
        colores = {inst: paleta[i % len(paleta)] for i, inst in enumerate(instrumentos)}

        # Configurar trazas según estilo
        for instrumento in instrumentos:
            df_instr = df_filtrado[df_filtrado['INSTRUMENTO'] == instrumento]
            color = colores[instrumento]

            config_base = {
                'x': df_instr['FECHA'],
                'y': df_instr[variable],
                'name': instrumento,
                'line': dict(width=grosor, color=color),
                'marker': dict(color=color)
            }

            if estilo == "Curvas suaves (spline)":
                fig.add_trace(go.Scatter(
                    mode="lines",
                    line_shape="spline",
                    **config_base
                ))
            elif estilo == "Líneas rectas":
                fig.add_trace(go.Scatter(
                    mode="lines",
                    line_shape="linear",
                    **config_base
                ))
            elif estilo == "Puntos":
                fig.add_trace(go.Scatter(
                    mode="markers",
                    **config_base
                ))
            elif estilo == "Líneas + Puntos":
                fig.add_trace(go.Scatter(
                    mode="lines+markers",
                    line_shape="linear",
                    **config_base
                ))
            elif estilo == "Área apilada":
                fig.add_trace(go.Scatter(
                    mode="lines",
                    stackgroup='one',
                    **config_base
                ))
            elif estilo == "Área + Líneas":
                fig.add_trace(go.Scatter(
                    mode="lines",
                    fill="tozeroy",
                    **config_base
                ))
            elif estilo == "Área + Líneas + Puntos":
                fig.add_trace(go.Scatter(
                    mode="lines+markers",
                    fill="tozeroy",
                    **config_base
                ))

        # Configuración de layout
        fig.update_layout(
            width=ancho,
            height=alto,
            title=dict(
                text=f"{titulo_dataset}: {variable} vs Tiempo",
                font=dict(size=18, color='#1e3c72')
            ),
            xaxis_title="Fecha",
            yaxis_title=variable,
            legend_title="Instrumentos",
            hovermode="x unified",
            plot_bgcolor='rgba(240, 244, 249, 1)',
            paper_bgcolor='rgba(255, 255, 255, 0.9)',
            margin=dict(l=50, r=50, t=80, b=50)
        )

        fig.show()

# --- Función de Exportación ---
def exportar_grafico(b=None):
    with output_mensajes:
        clear_output(wait=True)

        if fig is None:
            print("⚠️ Genere un gráfico antes de exportar")
            return

        extension = FORMATOS_GUARDADO[formato_dropdown.value]
        archivo = ruta_text.value + extension

        try:
            if formato_dropdown.value in ["PNG", "JPEG", "SVG", "PDF"]:
                fig.write_image(archivo)
                mensaje = f"✅ Gráfico exportado como imagen ({formato_dropdown.value})"
            elif formato_dropdown.value == "HTML":
                fig.write_html(archivo)
                mensaje = "✅ Gráfico exportado como HTML interactivo"

            print(mensaje)
            print(f"📁 Archivo: {os.path.abspath(archivo)}")

            # Descarga automática en Colab
            if 'google.colab' in str(get_ipython()):
                from google.colab import files
                files.download(archivo)

        except Exception as e:
            print(f"❌ Error en exportación: {str(e)}")
            if "kaleido" in str(e):
                print("ℹ️ Solución: Instale el motor de renderizado:\n!pip install -U kaleido")

# --- Asignación de Eventos ---
margen_dropdown.observe(actualizar_dataset, names='value')
boton_graficar.on_click(generar_grafico)
boton_guardar.on_click(exportar_grafico)

# --- Layout Final ---
controles_superiores = widgets.VBox([
    widgets.HBox([margen_dropdown, anio_dropdown]),
    widgets.HBox([punto_dropdown, variable_dropdown])
], layout=widgets.Layout(margin='0 0 20px 0'))

controles_estilo = widgets.HBox([
    estilo_dropdown,
    tamanio_dropdown,
    grosor_dropdown,
    paleta_dropdown
])

controles_guardado = widgets.HBox([
    formato_dropdown,
    ruta_text
])

botonera = widgets.HBox([
    boton_graficar,
    boton_guardar
], layout=widgets.Layout(justify_content='center', margin='15px 0'))

# --- Mostrar todos los componentes ---
display(widgets.VBox([
    widgets.HTML("<h3 style='color:#1e3c72; margin-top:20px'>Configuración de Análisis</h3>"),
    controles_superiores,
    controles_estilo,
    widgets.HTML("<h3 style='color:#1e3c72; margin-top:20px'>Exportación de Resultados</h3>"),
    controles_guardado,
    botonera,
    widgets.HTML("<div class='area-grafico'>"),
    output_grafico,
    widgets.HTML("</div>"),
    output_mensajes
]))

#Visualizaciones de los Puntos Fijos de Márgen Derecha / Izquierda (ChatGPT)

In [6]:
# === VISUALIZACIÓN INTERACTIVA UNIFICADA PARA PUNTOS FIJOS MI + MD ===

import pandas as pd
import plotly.graph_objects as go
import plotly.colors as pc
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import importlib.util
import os

# ======= VERIFICAR QUE LOS DATAFRAMES ESTÉN DEFINIDOS ========
if 'df_mi' not in globals() or 'df_md' not in globals():
    raise ValueError("❌ Asegúrate de haber cargado los DataFrames 'df_mi' y 'df_md' previamente.")

# Convertir fechas y preparar márgenes
for df in [df_mi, df_md]:
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')

# ======= SELECTOR DE MARGEN ========
margen_dropdown = widgets.Dropdown(
    options={"Margen Izquierda (MI)": "MI", "Margen Derecha (MD)": "MD"},
    value="MI",
    description="Margen:"
)

# Variables dinámicas (se actualizan luego)
punto_dropdown = widgets.Dropdown(description="Punto Fijo:")
variable_dropdown = widgets.Dropdown(description="Variable:")
anio_dropdown = widgets.Dropdown(description="Año:")

# Estilos y configuración gráfica
estilos_grafico = [
    "Curvas suaves (spline)",
    "Líneas rectas",
    "Puntos",
    "Líneas + Puntos",
    "Área apilada",
    "Área + Líneas",
    "Área + Líneas + Puntos"
]

tamanios_imagen = {
    "Pequeño (600x400)": (600, 400),
    "Mediano (900x500)": (900, 500),
    "Grande (1200x700)": (1200, 700),
    "Extra grande (1600x1000)": (1600, 1000)
}

grosores = {
    "Fino (1px)": 1,
    "Normal (2px)": 2,
    "Medio (4px)": 4,
    "Grueso (7px)": 7,
    "Extra grueso (10px)": 10
}

paletas = {
    "Plotly": pc.qualitative.Plotly,
    "D3": pc.qualitative.D3,
    "Viridis": pc.sequential.Viridis,
    "Cividis": pc.sequential.Cividis,
    "Inferno": pc.sequential.Inferno,
    "Pastel": pc.qualitative.Pastel,
    "Bold": pc.qualitative.Bold,
    "Set1": pc.qualitative.Set1,
    "Dark2": pc.qualitative.Dark2
}

# Selectores generales
estilo_dropdown = widgets.Dropdown(options=estilos_grafico, value="Curvas suaves (spline)", description="Estilo:")
tamanio_dropdown = widgets.Dropdown(options=tamanios_imagen.keys(), value="Mediano (900x500)", description="Tamaño:")
grosor_dropdown = widgets.Dropdown(options=grosores.keys(), value="Normal (2px)", description="Grosor:")
paleta_dropdown = widgets.Dropdown(options=paletas.keys(), value="Plotly", description="Colores:")
boton_graficar = widgets.Button(description="Graficar", button_style="success")
output = widgets.Output()

# Guardado
formatos = {"PNG": ".png", "JPEG": ".jpg", "SVG": ".svg", "PDF": ".pdf", "HTML": ".html"}
formato_dropdown = widgets.Dropdown(options=formatos.keys(), value="PNG", description="Formato:")
ruta_text = widgets.Text(value="grafica_exportada", description="Ruta:")
boton_guardar = widgets.Button(description="Guardar gráfica", button_style="info")
output_guardar = widgets.Output()

# Función para actualizar selectores según margen
def actualizar_selectores(*args):
    margen = margen_dropdown.value
    df = df_mi.copy() if margen == "MI" else df_md.copy()
    df = df.dropna(subset=['FECHA'])
    puntos = sorted(df['INSTRUMENTO'].dropna().unique())
    variables = [col for col in df.select_dtypes(include='number').columns if col not in ['FECHA', 'INSTRUMENTO', 'MARGEN']]
    anios = sorted(df['FECHA'].dt.year.dropna().unique())

    punto_dropdown.options = ["Todos"] + puntos
    punto_dropdown.value = "Todos"
    variable_dropdown.options = variables
    variable_dropdown.value = variables[0] if variables else None
    anio_dropdown.options = ["Todos"] + [str(a) for a in anios]
    anio_dropdown.value = "Todos"

margen_dropdown.observe(actualizar_selectores, names='value')
actualizar_selectores()

# Función para graficar
def graficar(b=None):
    global fig
    with output:
        clear_output(wait=True)
        df = df_mi.copy() if margen_dropdown.value == "MI" else df_md.copy()
        variable = variable_dropdown.value
        estilo = estilo_dropdown.value
        punto = punto_dropdown.value
        anio = anio_dropdown.value
        ancho, alto = tamanios_imagen[tamanio_dropdown.value]
        grosor = grosores[grosor_dropdown.value]
        paleta = paletas[paleta_dropdown.value]

        df = df.dropna(subset=['FECHA', 'INSTRUMENTO', variable])
        if anio != "Todos":
            df = df[df['FECHA'].dt.year == int(anio)]
        if punto != "Todos":
            df = df[df['INSTRUMENTO'] == punto]

        if df.empty:
            print("❌ No hay datos para graficar.")
            return

        fig = go.Figure()
        instrumentos = sorted(df['INSTRUMENTO'].unique())
        color_map = {pf: paleta[i % len(paleta)] for i, pf in enumerate(instrumentos)}

        for pf in instrumentos:
            data_pf = df[df['INSTRUMENTO'] == pf]
            line_args = dict(width=grosor, color=color_map[pf])
            marker_args = dict(color=color_map[pf])

            modo = {
                "Curvas suaves (spline)": ("lines", "spline"),
                "Líneas rectas": ("lines", "linear"),
                "Puntos": ("markers", None),
                "Líneas + Puntos": ("lines+markers", "linear"),
                "Área apilada": ("lines", "linear"),
                "Área + Líneas": ("lines", "linear"),
                "Área + Líneas + Puntos": ("lines+markers", "linear")
            }

            m, ls = modo[estilo]

            fig_args = dict(x=data_pf['FECHA'], y=data_pf[variable], name=pf, mode=m)
            if "Área" in estilo:
                fig_args["fill"] = "tozeroy" if "apilada" not in estilo else None
                fig_args["stackgroup"] = "one" if "apilada" in estilo else None

            if "lines" in m:
                fig_args["line"] = line_args
            if "markers" in m:
                fig_args["marker"] = marker_args
            if ls:
                fig_args["line_shape"] = ls

            fig.add_trace(go.Scatter(**fig_args))

        fig.update_layout(
            width=ancho,
            height=alto,
            title=f"Puntos Fijos Margen {margen_dropdown.value}: {variable}",
            xaxis_title="Fecha",
            yaxis_title=variable,
            legend_title="Instrumento",
            hovermode="x unified"
        )
        fig.show()

# Función para guardar
def guardar_grafica(b=None):
    with output_guardar:
        clear_output(wait=True)
        if 'fig' not in globals():
            print("❌ Primero debes generar una gráfica.")
            return
        ext = formatos[formato_dropdown.value]
        ruta = ruta_text.value
        if not ruta.endswith(ext):
            ruta += ext
        try:
            if formato_dropdown.value in ["PNG", "JPEG", "SVG", "PDF"]:
                if importlib.util.find_spec("kaleido") is None:
                    print("❌ Instala kaleido: %pip install -U kaleido")
                    return
                fig.write_image(ruta)
            elif formato_dropdown.value == "HTML":
                fig.write_html(ruta)
            print(f"✅ Gráfica guardada en: {os.path.abspath(ruta)}")
        except Exception as e:
            print("❌ Error al guardar:", e)

# Vínculo de botones
boton_graficar.on_click(graficar)
boton_guardar.on_click(guardar_grafica)

# === TÍTULO DE INTERFAZ ===
titulo_html = "<h2 style='color:#1a5276;'>Visualización de Puntos Fijos (MI y MD)</h2>"
display(HTML(titulo_html))

# === ORGANIZACIÓN VISUAL ===
display(widgets.VBox([
    margen_dropdown,
    widgets.HBox([punto_dropdown, variable_dropdown, anio_dropdown]),
    widgets.HBox([estilo_dropdown, tamanio_dropdown, grosor_dropdown, paleta_dropdown]),
    boton_graficar,
    output,
    widgets.HBox([formato_dropdown, ruta_text]),
    boton_guardar,
    output_guardar
]))


#Visualizaciones de los Puntos Fijos de Márgen Derecha / Izquierda (Grok)

In [8]:
# === VISUALIZACIÓN INTERACTIVA UNIFICADA PARA PUNTOS FIJOS DE REPRESAS PATAGONIA ===
# Este script unifica la visualización de datos de puntos fijos para Márgen Izquierda (MI) y Márgen Derecha (MD).
# Permite seleccionar el margen, punto fijo, variable, estilo gráfico, año, tamaño, grosor de línea y paleta de colores.
# Incluye opciones para guardar la gráfica en múltiples formatos (PNG, JPEG, SVG, PDF, HTML).
# Optimizado para Google Colab con un diseño visual atractivo e intuitivo.

import pandas as pd
import plotly.graph_objects as go
import plotly.colors as pc
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import importlib.util
import os

# --- VALIDACIÓN DE DEPENDENCIAS ---
try:
    import plotly.graph_objects
    import ipywidgets
except ImportError:
    raise ImportError("Ejecuta: %pip install plotly ipywidgets")

# --- VALIDACIÓN DE DATAFRAMES ---
try:
    df_mi
    df_md
except NameError:
    raise NameError("❌ Los DataFrames 'df_mi' o 'df_md' no están definidos. Por favor, ejecuta primero la carga de datos.")

# --- CONFIGURACIÓN INICIAL ---
# Definir opciones de configuración para gráficos
estilos_grafico = [
    "Curvas suaves (spline)",
    "Líneas rectas",
    "Puntos",
    "Líneas + Puntos",
    "Área apilada",
    "Área + Líneas",
    "Área + Líneas + Puntos"
]

tamanios_imagen = {
    "Pequeño (600x400)": (600, 400),
    "Mediano (900x500)": (900, 500),
    "Grande (1200x700)": (1200, 700),
    "Extra grande (1600x1000)": (1600, 1000)
}

grosores = {
    "Fino (1px)": 1,
    "Normal (2px)": 2,
    "Medio (4px)": 4,
    "Grueso (7px)": 7,
    "Extra grueso (10px)": 10
}

paletas = {
    "Plotly": pc.qualitative.Plotly,
    "D3": pc.qualitative.D3,
    "Viridis": pc.sequential.Viridis,
    "Cividis": pc.sequential.Cividis,
    "Inferno": pc.sequential.Inferno,
    "Pastel": pc.qualitative.Pastel,
    "Bold": pc.qualitative.Bold,
    "Set1": pc.qualitative.Set1,
    "Dark2": pc.qualitative.Dark2
}

formatos = {
    "PNG": ".png",
    "JPEG": ".jpg",
    "SVG": ".svg",
    "PDF": ".pdf",
    "HTML": ".html"
}

# --- FUNCIÓN PARA ACTUALIZAR OPCIONES DE PUNTOS Y VARIABLES ---
def actualizar_opciones(df):
    """Actualiza las opciones de puntos fijos y variables según el DataFrame seleccionado."""
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    columnas_excluidas = ['FECHA', 'INSTRUMENTO', 'MARGEN']
    variables = [col for col in df.select_dtypes(include='number').columns if col not in columnas_excluidas]
    puntos_fijos = sorted(df['INSTRUMENTO'].dropna().unique())
    opciones_puntos = ["Todos"] + list(puntos_fijos)
    anios = sorted(df['FECHA'].dt.year.dropna().unique())
    opciones_anios = ["Todos"] + [str(a) for a in anios]
    return variables, opciones_puntos, opciones_anios

# --- WIDGETS DE INTERFAZ ---
# Selector de margen
margen_dropdown = widgets.Dropdown(
    options=["Márgen Izquierda (MI)", "Márgen Derecha (MD)"],
    value="Márgen Izquierda (MI)",
    description="Margen:",
    style={'description_width': 'initial'},
    layout={'width': '250px'}
)

# Inicializar opciones para MI por defecto
variables, opciones_puntos, opciones_anios = actualizar_opciones(df_mi)

# Otros selectores
punto_dropdown = widgets.Dropdown(
    options=opciones_puntos,
    value="Todos",
    description="Punto Fijo:",
    style={'description_width': 'initial'},
    layout={'width': '250px'}
)
variable_dropdown = widgets.Dropdown(
    options=variables,
    description="Variable:",
    style={'description_width': 'initial'},
    layout={'width': '250px'}
)
estilo_dropdown = widgets.Dropdown(
    options=estilos_grafico,
    value="Curvas suaves (spline)",
    description="Estilo gráfica:",
    style={'description_width': 'initial'},
    layout={'width': '250px'}
)
anio_dropdown = widgets.Dropdown(
    options=opciones_anios,
    value="Todos",
    description="Año:",
    style={'description_width': 'initial'},
    layout={'width': '200px'}
)
tamanio_dropdown = widgets.Dropdown(
    options=list(tamanios_imagen.keys()),
    value="Mediano (900x500)",
    description="Tamaño:",
    style={'description_width': 'initial'},
    layout={'width': '250px'}
)
grosor_dropdown = widgets.Dropdown(
    options=list(grosores.keys()),
    value="Normal (2px)",
    description="Grosor línea:",
    style={'description_width': 'initial'},
    layout={'width': '200px'}
)
paleta_dropdown = widgets.Dropdown(
    options=list(paletas.keys()),
    value="Plotly",
    description="Paleta colores:",
    style={'description_width': 'initial'},
    layout={'width': '200px'}
)

# Botones
boton_graficar = widgets.Button(
    description="Graficar",
    button_style="success",
    tooltip="Generar la gráfica",
    layout={'width': '150px'}
)
formato_dropdown = widgets.Dropdown(
    options=list(formatos.keys()),
    value="PNG",
    description="Formato:",
    style={'description_width': 'initial'},
    layout={'width': '150px'}
)
ruta_text = widgets.Text(
    value="grafica_exportada",
    description="Ruta y nombre:",
    placeholder="ej: ./carpeta/mi_grafica",
    style={'description_width': 'initial'},
    layout={'width': '300px'}
)
boton_guardar = widgets.Button(
    description="Guardar gráfica",
    button_style="info",
    tooltip="Guardar la gráfica en el formato seleccionado",
    layout={'width': '150px'}
)

# Salidas
output = widgets.Output()
output_guardar = widgets.Output()

# --- FUNCIÓN PARA ACTUALIZAR OPCIONES AL CAMBIAR EL MARGEN ---
def on_margen_change(change):
    """Actualiza las opciones de puntos y variables al cambiar el margen."""
    with output:
        clear_output(wait=True)
        df = df_mi if change['new'] == "Márgen Izquierda (MI)" else df_md
        variables, opciones_puntos, opciones_anios = actualizar_opciones(df)
        punto_dropdown.options = opciones_puntos
        variable_dropdown.options = variables
        anio_dropdown.options = opciones_anios
        punto_dropdown.value = "Todos"
        variable_dropdown.value = variables[0] if variables else None
        anio_dropdown.value = "Todos"

margen_dropdown.observe(on_margen_change, names='value')

# --- FUNCIÓN PARA GRAFICAR ---
def graficar(b=None):
    """Genera la gráfica interactiva según las selecciones del usuario."""
    global fig
    with output:
        clear_output(wait=True)
        # Seleccionar DataFrame según el margen
        df = df_mi if margen_dropdown.value == "Márgen Izquierda (MI)" else df_md
        titulo_dataset = f"Puntos Fijos {margen_dropdown.value}"

        # Obtener parámetros de los selectores
        variable = variable_dropdown.value
        estilo = estilo_dropdown.value
        punto = punto_dropdown.value
        anio = anio_dropdown.value
        ancho, alto = tamanios_imagen[tamanio_dropdown.value]
        grosor = grosores[grosor_dropdown.value]
        paleta = paletas[paleta_dropdown.value]

        # Preparar datos para graficar
        df_plot = df.dropna(subset=['FECHA', 'INSTRUMENTO', variable])
        if anio != "Todos":
            df_plot = df_plot[df_plot['FECHA'].dt.year == int(anio)]
        if punto != "Todos":
            df_plot = df_plot[df_plot['INSTRUMENTO'] == punto]
        if df_plot.empty:
            print("⚠️ No hay datos para graficar con la selección actual.")
            return

        # Crear figura
        fig = go.Figure()
        instrumentos = sorted(df_plot['INSTRUMENTO'].unique())
        color_map = {pf: paleta[i % len(paleta)] for i, pf in enumerate(instrumentos)}

        # Generar trazas según el estilo seleccionado
        for pf in instrumentos:
            data_pf = df_plot[df_plot['INSTRUMENTO'] == pf]
            line_args = dict(width=grosor, color=color_map[pf])
            marker_args = dict(color=color_map[pf])
            trace_params = {
                'x': data_pf['FECHA'],
                'y': data_pf[variable],
                'name': pf,
                'line': line_args,
                'marker': marker_args
            }

            if estilo == "Curvas suaves (spline)":
                trace_params.update(mode="lines", line_shape="spline")
            elif estilo == "Líneas rectas":
                trace_params.update(mode="lines", line_shape="linear")
            elif estilo == "Puntos":
                trace_params.update(mode="markers")
            elif estilo == "Líneas + Puntos":
                trace_params.update(mode="lines+markers", line_shape="linear")
            elif estilo == "Área apilada":
                trace_params.update(mode="lines", stackgroup='one', line_shape="linear")
            elif estilo == "Área + Líneas":
                trace_params.update(mode="lines", fill="tozeroy", line_shape="linear")
            elif estilo == "Área + Líneas + Puntos":
                trace_params.update(mode="lines+markers", fill="tozeroy", line_shape="linear")

            fig.add_trace(go.Scatter(**trace_params))

        # Configurar diseño de la gráfica
        fig.update_layout(
            width=ancho,
            height=alto,
            title=f"{titulo_dataset}: {variable} en función del tiempo por Punto Fijo (PF)",
            xaxis_title="Fecha",
            yaxis_title=variable,
            legend_title="INSTRUMENTO",
            hovermode="x unified",
            template="plotly_white",
            font=dict(family="Arial", size=12),
            margin=dict(l=50, r=50, t=100, b=50)
        )
        fig.show()

# --- FUNCIÓN PARA GUARDAR LA GRÁFICA ---
def guardar_grafica(b=None):
    """Guarda la gráfica en el formato y ruta especificados."""
    with output_guardar:
        clear_output(wait=True)
        ext = formatos[formato_dropdown.value]
        ruta_archivo = ruta_text.value
        if not ruta_archivo.lower().endswith(ext):
            ruta_archivo += ext
        if 'fig' not in globals() or not isinstance(fig, go.Figure):
            print("❌ Primero debes generar una gráfica.")
            return
        try:
            if formato_dropdown.value in ["PNG", "JPEG", "SVG", "PDF"]:
                if importlib.util.find_spec("kaleido") is None:
                    print("❌ Para guardar como imagen/vector/pdf, instala 'kaleido':\n%pip install -U kaleido")
                    return
                fig.write_image(ruta_archivo, format=formato_dropdown.value.lower())
            elif formato_dropdown.value == "HTML":
                fig.write_html(ruta_archivo)
            else:
                print("❌ Tipo de archivo no soportado.")
                return
            print(f"✅ Gráfica guardada en: {os.path.abspath(ruta_archivo)}")
            if os.path.exists(ruta_archivo) and 'google.colab' in str(get_ipython()):
                from google.colab import files
                files.download(ruta_archivo)
            else:
                ruta_abs = os.path.abspath(ruta_archivo)
                display(HTML(f'<a href="file://{ruta_abs}" target="_blank">Descargar archivo</a>'))
        except Exception as e:
            print(f"❌ Error al guardar la gráfica: {e}")

# --- ASIGNAR EVENTOS A BOTONES ---
boton_graficar.on_click(graficar)
boton_guardar.on_click(guardar_grafica)

# --- ORGANIZACIÓN VISUAL DE LA INTERFAZ ---
# Título principal
titulo_html = f"<h2 style='color:#1866a3; text-align:center; margin-bottom:10px'>Visualización de Puntos Fijos - Represas Patagonia</h2>"
display(HTML(titulo_html))

# Contenedor de selectores (dividido en dos filas para mejor organización)
selectores_fila1 = widgets.HBox(
    [margen_dropdown, punto_dropdown, variable_dropdown, estilo_dropdown],
    layout={'justify_content': 'space-between', 'margin': '10px 0'}
)
selectores_fila2 = widgets.HBox(
    [anio_dropdown, tamanio_dropdown, grosor_dropdown, paleta_dropdown],
    layout={'justify_content': 'space-between', 'margin': '10px 0'}
)

# Contenedor de botones y guardado
controles_guardar = widgets.HBox(
    [formato_dropdown, ruta_text, boton_guardar],
    layout={'justify_content': 'center', 'margin': '10px 0'}
)

# Mostrar elementos de la interfaz
display(selectores_fila1)
display(selectores_fila2)
display(widgets.HBox([boton_graficar], layout={'justify_content': 'center', 'margin': '10px 0'}))
display(output)
display(HTML("<h3 style='color:#1866a3; margin-top:20px'>Opciones de Guardado</h3>"))
display(controles_guardar)
display(output_guardar)

Output()

Output()

#Geminis

In [10]:
# === VISUALIZACIÓN INTERACTIVA UNIFICADA PARA PUNTOS FIJOS (MI y MD) ===

import pandas as pd
import plotly.graph_objects as go
import plotly.colors as pc
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import importlib.util
import os

# --- Verificación de DataFrames globales ---
if 'df_mi' not in globals() or 'df_md' not in globals() or df_mi.empty and df_md.empty:
    print("❌ No se han cargado los DataFrames 'df_mi' o 'df_md'.")
    print("Por favor, asegúrate de ejecutar el script de carga de archivos primero y subir los datos.")
else:
    # --- Controles de la Interfaz ---
    # Selector de Margen (MI o MD)
    margen_dropdown = widgets.Dropdown(
        options=['MI', 'MD'] if not df_mi.empty and not df_md.empty else (['MI'] if not df_mi.empty else ['MD']),
        description="Márgen:",
        value='MI' if not df_mi.empty else ('MD' if not df_md.empty else None) # Valor por defecto
    )

    # Inicialización de opciones (se actualizarán al cambiar el margen)
    punto_dropdown = widgets.Dropdown(description="Punto Fijo:")
    variable_dropdown = widgets.Dropdown(description="Variable:")
    anio_dropdown = widgets.Dropdown(description="Año:")

    estilos_grafico = [
        "Curvas suaves (spline)",
        "Líneas rectas",
        "Puntos",
        "Líneas + Puntos",
        "Área apilada",
        "Área + Líneas",
        "Área + Líneas + Puntos"
    ]
    estilo_dropdown = widgets.Dropdown(
        options=estilos_grafico,
        value="Curvas suaves (spline)",
        description="Estilo gráfica:"
    )

    tamanios_imagen = {
        "Pequeño (600x400)": (600, 400),
        "Mediano (900x500)": (900, 500),
        "Grande (1200x700)": (1200, 700),
        "Extra grande (1600x1000)": (1600, 1000)
    }
    tamanio_dropdown = widgets.Dropdown(
        options=list(tamanios_imagen.keys()),
        value="Mediano (900x500)",
        description="Tamaño:"
    )

    grosores = {
        "Fino (1px)": 1,
        "Normal (2px)": 2,
        "Medio (4px)": 4,
        "Grueso (7px)": 7,
        "Extra grueso (10px)": 10
    }
    grosor_dropdown = widgets.Dropdown(
        options=list(grosores.keys()),
        value="Normal (2px)",
        description="Grosor línea:"
    )

    paletas = {
        "Plotly": pc.qualitative.Plotly,
        "D3": pc.qualitative.D3,
        "Viridis": pc.sequential.Viridis,
        "Cividis": pc.sequential.Cividis,
        "Inferno": pc.sequential.Inferno,
        "Pastel": pc.qualitative.Pastel,
        "Bold": pc.qualitative.Bold,
        "Set1": pc.qualitative.Set1,
        "Dark2": pc.qualitative.Dark2
    }
    paleta_dropdown = widgets.Dropdown(
        options=list(paletas.keys()),
        value="Plotly",
        description="Paleta colores:"
    )

    boton_graficar = widgets.Button(description="Graficar", button_style="success", icon='chart-line')
    output_grafica = widgets.Output()
    output_guardar = widgets.Output()

    # --- Bloque para Guardar la Gráfica ---
    formatos = {
        "PNG": ".png",
        "JPEG": ".jpg",
        "SVG": ".svg",
        "PDF": ".pdf",
        "HTML": ".html"
    }

    formato_dropdown = widgets.Dropdown(
        options=list(formatos.keys()),
        value="PNG",
        description="Formato:"
    )
    ruta_text = widgets.Text(
        value="grafica_exportada",
        description="Ruta y nombre:",
        placeholder="ej: ./carpeta/mi_grafica"
    )
    boton_guardar = widgets.Button(description="Guardar Gráfica", button_style="info", icon='save')

    def guardar_grafica(b=None):
        """Guarda la gráfica generada en el formato y ruta especificados."""
        with output_guardar:
            clear_output(wait=True)
            ext = formatos[formato_dropdown.value]
            ruta_archivo = ruta_text.value
            if not ruta_archivo.lower().endswith(ext):
                ruta_archivo += ext

            if 'fig' not in globals() or not isinstance(fig, go.Figure):
                print("❌ Primero debes generar una gráfica para poder guardarla.")
                return
            try:
                if formato_dropdown.value in ["PNG", "JPEG", "SVG", "PDF"]:
                    # Verifica si kaleido está instalado para exportar imágenes/PDF
                    if importlib.util.find_spec("kaleido") is None:
                        print("❌ Para guardar como imagen/vector/pdf, instala 'kaleido':\n%pip install -U kaleido")
                        return
                    fig.write_image(ruta_archivo, format=formato_dropdown.value.lower())
                elif formato_dropdown.value == "HTML":
                    fig.write_html(ruta_archivo)
                else:
                    print("❌ Tipo de archivo no soportado.")
                    return
            except Exception as e:
                print(f"❌ Error al guardar la gráfica: {e}")
                return

            print(f"✅ Gráfica guardada en: {os.path.abspath(ruta_archivo)}")
            # Ofrece la descarga si estamos en Google Colab
            if os.path.exists(ruta_archivo):
                if 'google.colab' in str(get_ipython()):
                    from google.colab import files
                    files.download(ruta_archivo)
                else:
                    ruta_abs = os.path.abspath(ruta_archivo)
                    display(HTML(f'<a href="file://{ruta_abs}" target="_blank">Descargar archivo</a>'))

    # --- Función para actualizar las opciones de los dropdowns al cambiar el margen ---
    def actualizar_opciones(change):
        """Actualiza las opciones de Punto Fijo, Variable y Año según el margen seleccionado."""
        global df_actual
        selected_margen = margen_dropdown.value
        if selected_margen == 'MI':
            df_actual = df_mi.copy()
            titulo_dataset = "Puntos Fijos Márgen Izquierda (MI)"
        else: # selected_margen == 'MD'
            df_actual = df_md.copy()
            titulo_dataset = "Puntos Fijos Márgen Derecha (MD)"

        # Asegúrate de que 'FECHA' es datetime
        df_actual['FECHA'] = pd.to_datetime(df_actual['FECHA'], dayfirst=True, errors='coerce')

        columnas_excluidas = ['FECHA', 'INSTRUMENTO', 'MARGEN'] # 'MARGEN' podría no existir en los CSV originales
        variables = [col for col in df_actual.select_dtypes(include='number').columns if col not in columnas_excluidas]
        puntos_fijos = sorted(df_actual['INSTRUMENTO'].dropna().unique())
        anios = sorted(df_actual['FECHA'].dt.year.dropna().unique())

        # Actualiza las opciones de los dropdowns
        with punto_dropdown.hold_sync():
            punto_dropdown.options = ["Todos"] + list(puntos_fijos)
            punto_dropdown.value = "Todos" if "Todos" in punto_dropdown.options else (list(puntos_fijos)[0] if puntos_fijos else None)

        with variable_dropdown.hold_sync():
            variable_dropdown.options = variables
            variable_dropdown.value = variables[0] if variables else None

        with anio_dropdown.hold_sync():
            anio_dropdown.options = ["Todos"] + [str(a) for a in anios]
            anio_dropdown.value = "Todos" if "Todos" in anio_dropdown.options else (str(anios[0]) if anios else None)

        # Actualiza el título del dataset para reflejar el margen actual
        titulo_html.value = f"<h2 style='color:#1866a3; margin-bottom: 5px'>{titulo_dataset}</h2>"

        # Borra la gráfica y el mensaje de guardado al cambiar de margen
        with output_grafica:
            clear_output(wait=True)
        with output_guardar:
            clear_output(wait=True)

    # --- Función principal para graficar ---
    def graficar(b=None):
        """Genera la gráfica interactiva basada en las selecciones del usuario."""
        global fig # Hacemos 'fig' global para que la función de guardar pueda acceder a ella
        with output_grafica:
            clear_output(wait=True) # Limpia la salida antes de graficar

            variable = variable_dropdown.value
            estilo = estilo_dropdown.value
            punto = punto_dropdown.value
            anio = anio_dropdown.value
            ancho, alto = tamanios_imagen[tamanio_dropdown.value]
            grosor = grosores[grosor_dropdown.value]
            paleta = paletas[paleta_dropdown.value]

            # Filtra el DataFrame según las selecciones
            df_plot = df_actual.dropna(subset=['FECHA', 'INSTRUMENTO', variable])

            if anio != "Todos":
                df_plot = df_plot[df_plot['FECHA'].dt.year == int(anio)]
            if punto != "Todos":
                df_plot = df_plot[df_plot['INSTRUMENTO'] == punto]

            if df_plot.empty:
                print("No hay datos para graficar con la selección actual. Intenta ajustar los filtros.")
                return

            fig = go.Figure()
            instrumentos = sorted(df_plot['INSTRUMENTO'].unique())
            color_map = {pf: paleta[i % len(paleta)] for i, pf in enumerate(instrumentos)}

            # Añade trazas para cada instrumento según el estilo de gráfica seleccionado
            for pf in instrumentos:
                data_pf = df_plot[df_plot['INSTRUMENTO'] == pf]
                line_args = dict(width=grosor, color=color_map[pf])
                marker_args = dict(color=color_map[pf])

                # Determina el modo y line_shape según el estilo
                mode = ""
                line_shape = "linear"
                fill_mode = "none" # Por defecto no hay relleno
                if "Líneas" in estilo:
                    mode += "lines"
                    if "suaves" in estilo:
                        line_shape = "spline"
                if "Puntos" in estilo:
                    mode += "+markers"
                if "Área" in estilo:
                    fill_mode = "tozeroy"
                    if "apilada" in estilo:
                        stackgroup = 'one'
                    else:
                        stackgroup = None
                else:
                    stackgroup = None # Asegurarse de que no se aplica stackgroup si no es área apilada

                # Asegurarse de que 'mode' no esté vacío si no es área
                if not mode and "Área" not in estilo:
                    mode = "lines" # Comportamiento por defecto si no se especifican líneas ni puntos

                fig.add_trace(go.Scatter(
                    x=data_pf['FECHA'],
                    y=data_pf[variable],
                    mode=mode,
                    name=pf,
                    line_shape=line_shape,
                    line=line_args,
                    marker=marker_args,
                    fill=fill_mode,
                    stackgroup=stackgroup # Aplica stackgroup solo si es área apilada
                ))

            # Actualiza el layout de la gráfica
            fig.update_layout(
                width=ancho,
                height=alto,
                title=f"{titulo_dataset}: {variable} en función del tiempo por Punto Fijo (PF)",
                xaxis_title="Fecha",
                yaxis_title=variable,
                legend_title="INSTRUMENTO",
                hovermode="x unified",
                template="plotly_white" # Un template limpio y profesional
            )
            fig.show()

    # --- Enlace de funciones a eventos de widgets ---
    margen_dropdown.observe(actualizar_opciones, names='value')
    boton_graficar.on_click(graficar)
    boton_guardar.on_click(guardar_grafica)

    # --- Configuración inicial de opciones al cargar la visualización ---
    df_actual = pd.DataFrame() # DataFrame que se usará para la visualización activa
    titulo_dataset = "" # Título que se actualizará

    # Crea un HBox para el título actualizable
    titulo_html = widgets.HTML()

    # Llama a actualizar_opciones una vez para inicializar los dropdowns con el valor predeterminado del margen
    actualizar_opciones(None) # Llama con None para la primera inicialización

    # --- Diseño de la Interfaz ---
    # Título principal
    display(HTML("<h1 style='color:#0d47a1; text-align:center;'>Análisis de Puntos Fijos en Presas Patagonia</h1>"))

    # Controles de selección de datos y filtros
    controles_datos = widgets.VBox([
        margen_dropdown,
        widgets.HBox([punto_dropdown, variable_dropdown, anio_dropdown])
    ], layout=widgets.Layout(border='2px solid #ccc', padding='10px', margin='10px 0'))

    # Controles de estilo de gráfica
    controles_estilo = widgets.VBox([
        estilo_dropdown,
        widgets.HBox([tamanio_dropdown, grosor_dropdown, paleta_dropdown])
    ], layout=widgets.Layout(border='2px solid #ccc', padding='10px', margin='10px 0'))

    # Botón de graficar
    botonera_principal = widgets.HBox([
        boton_graficar
    ], layout=widgets.Layout(justify_content='center', margin='10px 0'))

    # Controles de guardado
    controles_guardado = widgets.VBox([
        widgets.HTML("<h3 style='color:#0d47a1;'>Exportar Gráfica</h3>"),
        widgets.HBox([formato_dropdown, ruta_text]),
        boton_guardar
    ], layout=widgets.Layout(border='2px solid #ccc', padding='10px', margin='10px 0'))

    # Layout general de la interfaz
    display(
        titulo_html,
        controles_datos,
        controles_estilo,
        botonera_principal,
        output_grafica,
        controles_guardado,
        output_guardar
    )

HTML(value="<h2 style='color:#1866a3; margin-bottom: 5px'>Puntos Fijos Márgen Izquierda (MI)</h2>")

Output()

Output()

# 🧠 GENERACIÓN DE ALARMAS EN DATOS DE AUSCULTACIÓN

In [24]:
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import plotly.graph_objects as go
from datetime import datetime

# --- ESTILOS CSS ---
estilo_css = """
<style>
.widget-label { font-weight: bold !important; color: #1e3c72 !important; }
.alert-critical { color: red; font-weight: bold; }
.alert-warning { color: orange; font-weight: bold; }
.alert-info { color: #0066cc; font-weight: bold; }
.alert-normal { color: green; font-weight: bold; }
</style>
"""
display(HTML(estilo_css))

# --- PREPARACIÓN DE DATOS ---
def preparar_datos(df_mi, df_md):
    """Prepara los dataframes para análisis"""
    df_mi = df_mi.copy()
    df_md = df_md.copy()

    # Limpieza de columnas y fechas
    for df in [df_mi, df_md]:
        df.columns = df.columns.str.strip()
        df['FECHA'] = pd.to_datetime(df['FECHA'], errors='coerce')
        df.dropna(subset=['FECHA'], inplace=True)

    return df_mi, df_md

df_mi, df_md = preparar_datos(df_mi, df_md)

# --- WIDGETS INTERACTIVOS ---
# Selector de margen
margen_selector = widgets.Dropdown(
    options=['Margen Izquierda (MI)', 'Margen Derecha (MD)'],
    value='Margen Izquierda (MI)',
    description='Márgen:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

# Selector de punto fijo
punto_selector = widgets.Dropdown(
    options=sorted(df_mi['INSTRUMENTO'].unique()),
    description='Punto fijo:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

# Selector de variable
variable_selector = widgets.Dropdown(
    options=[col for col in df_mi.select_dtypes(include='number').columns if col != 'FECHA'],
    description='Variable:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

# Selector de umbrales
umbral_selector = widgets.Dropdown(
    options=[
        ('Automático (recomendado)', 'auto'),
        ('Personalizado', 'custom'),
        ('Normativa ICOLD', 'icold'),
        ('Sensible (umbral bajo)', 'low'),
        ('Conservador (umbral alto)', 'high')
    ],
    value='auto',
    description='Umbral alertas:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='300px')
)

# Botones
boton_analizar = widgets.Button(
    description="Analizar Datos",
    button_style='success',
    icon='search',
    layout=widgets.Layout(width='200px')
)

boton_graficar = widgets.Button(
    description="Visualizar Tendencias",
    button_style='info',
    icon='chart-line',
    layout=widgets.Layout(width='200px')
)

# --- FUNCIONES DE ACTUALIZACIÓN ---
def actualizar_selectores(change):
    """Actualiza los selectores según el margen seleccionado"""
    if margen_selector.value == 'Margen Izquierda (MI)':
        df = df_mi
    else:
        df = df_md

    # Actualizar opciones
    puntos = sorted(df['INSTRUMENTO'].dropna().unique())
    variables = [col for col in df.select_dtypes(include='number').columns if col != 'FECHA']

    punto_selector.options = puntos
    variable_selector.options = variables

margen_selector.observe(actualizar_selectores, names='value')

# --- FUNCIÓN DE ANÁLISIS MEJORADA ---
def analizar_variaciones(df, variable, umbral_mode='auto'):
    """Realiza análisis completo de variaciones con sistema de alarmas"""
    df = df.sort_values('FECHA').copy()

    # Cálculos básicos
    df['valor_anterior'] = df[variable].shift(1)
    df['delta'] = df[variable] - df['valor_anterior']
    df['dias_entre_mediciones'] = (df['FECHA'] - df['FECHA'].shift(1)).dt.days

    # Calcular velocidad (mm/día)
    df['velocidad'] = np.where(
        df['dias_entre_mediciones'] > 0,
        df['delta'] / df['dias_entre_mediciones'],
        np.nan
    )

    # Determinar umbrales según modo seleccionado
    if umbral_mode == 'auto':
        # Umbrales automáticos basados en percentiles
        abs_deltas = df['delta'].abs().dropna()
        if not abs_deltas.empty:
            p75 = abs_deltas.quantile(0.75)
            p90 = abs_deltas.quantile(0.90)
            umbral_precaucion = max(2, p75)
            umbral_alerta = max(5, p90)
            umbral_critico = max(10, p90 * 1.5)
        else:
            umbral_precaucion, umbral_alerta, umbral_critico = 2, 5, 10
    elif umbral_mode == 'icold':
        umbral_precaucion, umbral_alerta, umbral_critico = 3, 6, 10
    elif umbral_mode == 'low':
        umbral_precaucion, umbral_alerta, umbral_critico = 1, 3, 5
    elif umbral_mode == 'high':
        umbral_precaucion, umbral_alerta, umbral_critico = 5, 10, 15
    else:  # custom
        umbral_precaucion, umbral_alerta, umbral_critico = 3, 6, 10

    # Sistema de alarmas mejorado
    conditions = [
        (df['delta'].abs() > umbral_critico) | (df['velocidad'].abs() > 0.5),
        (df['delta'].abs() > umbral_alerta) | (df['velocidad'].abs() > 0.3),
        (df['delta'].abs() > umbral_precaucion),
        df[variable].isna() | (df['dias_entre_mediciones'] > 90)
    ]

    choices = [
        f'🛑 CRÍTICO (> {umbral_critico}mm o >0.5mm/día)',
        f'🔴 ALERTA (> {umbral_alerta}mm o >0.3mm/día)',
        f'⚠️ PRECAUCIÓN (> {umbral_precaucion}mm)',
        '🔵 DATO FALTANTE/intervalo largo'
    ]

    df['ALARMA'] = np.select(conditions, choices, default='✅ NORMAL')

    # Análisis de tendencia
    df['tendencia'] = df[variable].rolling(window=3).mean().diff().apply(
        lambda x: '↑↑' if x > 0.1 else ('↓↓' if x < -0.1 else '→')
    )

    return df, {
        'umbral_precaucion': umbral_precaucion,
        'umbral_alerta': umbral_alerta,
        'umbral_critico': umbral_critico
    }

# --- FUNCIONES DE VISUALIZACIÓN ---
def mostrar_resultados(b):
    with output:
        clear_output(wait=True)

        margen = 'MI' if 'Izquierda' in margen_selector.value else 'MD'
        df = df_mi if margen == 'MI' else df_md
        punto = punto_selector.value
        variable = variable_selector.value
        umbral_mode = umbral_selector.value

        # Filtrar y analizar datos
        df_filtrado = df[df['INSTRUMENTO'] == punto].copy()
        df_analizado, umbrales = analizar_variaciones(df_filtrado, variable, umbral_mode)

        # Mostrar resumen
        display(HTML(f"""
        <h3 style='color:#1e3c72'>Análisis de {variable} - Punto {punto} ({margen})</h3>
        <p><b>Umbrales aplicados:</b> Precaución: {umbrales['umbral_precaucion']:.2f}mm |
        Alerta: {umbrales['umbral_alerta']:.2f}mm | Crítico: {umbrales['umbral_critico']:.2f}mm</p>
        """))

        # Mostrar tabla con las columnas más relevantes
        columnas_mostrar = [
            'FECHA', variable, 'valor_anterior', 'delta',
            'dias_entre_mediciones', 'velocidad', 'tendencia', 'ALARMA'
        ]

        # Formatear tabla para mejor visualización
        def color_alarma(val):
            if '🛑' in str(val):
                return 'background-color: #ffcccc'
            elif '🔴' in str(val):
                return 'background-color: #ffe6cc'
            elif '⚠️' in str(val):
                return 'background-color: #ffffcc'
            else:
                return ''

        styled_df = df_analizado[columnas_mostrar].style.applymap(color_alarma, subset=['ALARMA'])
        styled_df = styled_df.format({
            variable: '{:.2f}',
            'valor_anterior': '{:.2f}',
            'delta': '{:.2f}',
            'velocidad': '{:.4f}'
        })

        display(styled_df)

def graficar_tendencias(b):
    with output:
        clear_output(wait=True)

        margen = 'MI' if 'Izquierda' in margen_selector.value else 'MD'
        df = df_mi if margen == 'MI' else df_md
        punto = punto_selector.value
        variable = variable_selector.value

        df_filtrado = df[df['INSTRUMENTO'] == punto].sort_values('FECHA')

        # Crear gráfico interactivo con Plotly
        fig = go.Figure()

        # Línea principal
        fig.add_trace(go.Scatter(
            x=df_filtrado['FECHA'],
            y=df_filtrado[variable],
            mode='lines+markers',
            name=variable,
            line=dict(color='#1f77b4', width=2),
            marker=dict(size=8)
         ))

        # Añadir alarmas si existen
        if 'ALARMA' in df_filtrado.columns:
            alarmas_criticas = df_filtrado[df_filtrado['ALARMA'].str.contains('🛑')]
            if not alarmas_criticas.empty:
                fig.add_trace(go.Scatter(
                    x=alarmas_criticas['FECHA'],
                    y=alarmas_criticas[variable],
                    mode='markers',
                    name='Alerta crítica',
                    marker=dict(color='red', size=12, symbol='x')
                ))

        # Configuración del layout
        fig.update_layout(
            title=f'Tendencia de {variable} - Punto {punto} ({margen})',
            xaxis_title='Fecha',
            yaxis_title=variable,
            hovermode='x unified',
            template='plotly_white',
            height=600
        )

        fig.show()

# --- ASIGNACIÓN DE EVENTOS ---
boton_analizar.on_click(mostrar_resultados)
boton_graficar.on_click(graficar_tendencias)

# --- DISPOSICIÓN DE LA INTERFAZ ---
# Cabecera
display(HTML("<h2 style='color:#1e3c72'>Sistema de Monitoreo de Presas Patagonia</h2>"))

# Controles principales
controles_superiores = widgets.HBox([
    margen_selector,
    punto_selector,
    variable_selector,
    umbral_selector
], layout=widgets.Layout(justify_content='space-between'))

# Botones
controles_botones = widgets.HBox([
    boton_analizar,
    boton_graficar
], layout=widgets.Layout(justify_content='center', margin='20px 0'))

# Área de resultados
output = widgets.Output()

# Mostrar todos los componentes
display(widgets.VBox([
    controles_superiores,
    controles_botones,
    output
]))